### Multiple Parallel Input and Multi-step output

In [32]:
# multivariate output stacked lstm example
from numpy import array
from numpy import hstack, vstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers.core import Dropout
from sklearn.preprocessing import MinMaxScaler
from pandas import read_csv
from pandas import DataFrame
import matplotlib.pyplot as plt
import numpy as np


# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence

# Read the data
data = read_csv('pollution2.csv', header=0, index_col=0)

# horizontally stack columns
raw_data = data.values
print(f'The shape of the raw_data before scaling is {raw_data.shape}')
#normalize input features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(raw_data)

n_train = 2000
#dataset = raw_data[0:n_train,:]
dataset = scaled_data[0:n_train,:]
# choose a number of time steps
n_steps_in, n_steps_out = 30, 10
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=100, verbose=0)

# demonstrate iterative prediction
# x_inp = raw_data[n_train:n_train+n_steps,:]
x_inp = scaled_data[n_train:n_train+n_steps_in,:]
print(f'Initial shape of x_input {x_inp.shape}')
x_input = x_inp.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(f'The shape of yhat after prediction is {yhat.shape}')
yhat = yhat.reshape(n_steps_out, n_features)
print(f'The new shape of yhat  is {yhat.shape}')
inv_yhat = scaler.inverse_transform(yhat)
print(inv_yhat)


The shape of the raw_data before scaling is (43800, 5)
Initial shape of x_input (30, 5)
The shape of yhat after prediction is (1, 10, 5)
The new shape of yhat  is (10, 5)
[[ 4.6747547e+01 -1.3420012e+01  1.0151584e+01  1.0268147e+03
   3.9381290e+01]
 [ 4.7069244e+01 -1.4544997e+01  9.1466093e+00  1.0271481e+03
   3.6655029e+01]
 [ 5.4075783e+01 -1.3254982e+01  8.0811481e+00  1.0280106e+03
   3.2775898e+01]
 [ 6.3234592e+01 -1.1975583e+01  6.5521917e+00  1.0286493e+03
   2.9105993e+01]
 [ 7.0404572e+01 -1.0762006e+01  5.1529436e+00  1.0292001e+03
   2.4956392e+01]
 [ 7.5520630e+01 -9.7187052e+00  3.9584653e+00  1.0296611e+03
   2.0800138e+01]
 [ 7.9011429e+01 -8.8733530e+00  2.9694922e+00  1.0300353e+03
   1.7015844e+01]
 [ 8.1366043e+01 -8.2160168e+00  2.1623552e+00  1.0303318e+03
   1.3902097e+01]
 [ 8.2883957e+01 -7.7209105e+00  1.5108942e+00  1.0305634e+03
   1.1504594e+01]
 [ 8.3763771e+01 -7.3546691e+00  9.9416786e-01  1.0307437e+03
   9.6357288e+00]]


In [9]:
# Iterative execution